# k-nn

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import Normalizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [2]:
df = pd.read_csv("preprocessed data/fraud_oracle_preprocessed_drop.csv")
df.drop(['Unnamed: 0'], axis=1, inplace = True)
original_fraud_df = df

In [3]:
def get_translator(data):
    translator = {}
    for col in data.columns:
        old_values = data[col].unique().tolist()
        new_values = [1 + i for i in range(len(data[col].unique()))]
        if (data[col].dtypes == object):
            # data_profileprint(old_values, new_values)
            translator[col] = [old_values, new_values]
        else:
            translator[col] = [old_values, old_values[:]]
    return translator

def change_values(data, translator, flag=0):
    for col in data.columns:
        data[col] = data[col].replace(translator[col][flag + 0],
                                      translator[col][1 - flag])
    return data

translator = get_translator(df)
df = change_values(df, translator)

In [4]:
# labels = df.FraudFound_P
# data = df.drop(['FraudFound_P'], axis=1)

In [5]:

transformer = Normalizer().fit(df)  # fit does nothing.
transformer
Normalizer()
data = pd.DataFrame(transformer.transform(df))


In [6]:
# res = train_test_split(data, labels, 
#                        train_size=0.85,
#                        test_size=0.15,
#                        random_state=1200)
# train_data, test_data, train_labels, test_labels = res 

In [7]:
def get_results(k, train_data, train_labels, test_data, test_labels):
    knn = KNeighborsClassifier(n_neighbors = k, metric = 'cosine')
    knn.fit(train_data, train_labels)
    test_data_predicted = knn.predict(test_data)
    copy = test_labels.to_frame()
    result = copy.assign(predict = pd.DataFrame(test_data_predicted)[0].tolist())
    return result

In [8]:
TPR = [0,0,0,0,0,0,0,0,0,0]
FPR = [0,0,0,0,0,0,0,0,0,0]
ACC = []
x = []
for i in range(10):
    skf = StratifiedKFold(n_splits=3)
    skf.get_n_splits(df, df.FraudFound_P)
    x.append(i + 1)
    print(skf)
    StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
    for train_index, test_index in skf.split(df, df.FraudFound_P):
        train_data = df.loc[train_index,:]
        test_data = df.loc[test_index,:]
    
       
        result = get_results(i + 1, train_data.drop(['FraudFound_P'], axis=1), train_data.FraudFound_P, test_data.drop(['FraudFound_P'], axis=1), test_data.FraudFound_P)
        a = result[(result.predict == 1) & (result.FraudFound_P == 1)].count()[0]
        b = result[(result.predict == 0) & (result.FraudFound_P == 1)].count()[0]
        c = result[(result.predict == 1) & (result.FraudFound_P == 0)].count()[0]
        d = result[(result.predict == 0) & (result.FraudFound_P == 0)].count()[0]
        tab = [[a, c], [b, d]]
        print(tab)
        tpr = tab[0][0] / (tab[0][0] + tab[1][0])
        fpr = tab[0][1] / (tab[0][1] + tab[1][1])
        accuracy = (tab[0][0] + tab[1][1]) / (tab[0][0] + tab[1][0] + tab[1][1] +
                                              tab[0][1])
        TPR[i] += tpr/3
        FPR[i] += fpr/3
        ACC.append(accuracy)
    


StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
[[28, 256], [280, 4576]]
[[22, 252], [286, 4580]]
[[29, 267], [278, 4565]]
StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
[[4, 20], [304, 4812]]
[[3, 20], [305, 4812]]
[[4, 20], [303, 4812]]
StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
[[13, 57], [295, 4775]]
[[5, 68], [303, 4764]]
[[6, 55], [301, 4777]]
StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
[[2, 5], [306, 4827]]
[[3, 7], [305, 4825]]
[[1, 5], [306, 4827]]
StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
[[3, 12], [305, 4820]]
[[6, 17], [302, 4815]]
[[1, 12], [306, 4820]]
StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
[[1, 0], [307, 4832]]
[[3, 2], [305, 4830]]
[[0, 1], [307, 4831]]
StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
[[2, 4], [306, 4828]]
[[6, 5], [302, 4827]]
[[2, 5], [305, 4827]]
StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
[[0, 0], [308, 4832]]


KeyboardInterrupt: 

In [ ]:
plt.plot(x,TPR)
plt.plot(x,FPR)
plt.show()